<a target="_blank" href="https://colab.research.google.com/github/sudarshan-koirala/langchain-falcon-chainlit/blob/main/langchain_falcon.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [7]:
# USE CASE:    CONNECTED
#   TEXT  (YouTube Example = https://www.youtube.com/watch?v=gnyUUY8X-G4)   
# + VIDEO (https://huggingface.co/learn/audio-course/chapter6/pre-trained_models)

%%capture
%pip install langchain huggingface_hub watermark
%pip install transformers datasets soundfile speechbrain accelerate
%pip install sentencepiece
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [8]:
#%load_ext watermark
%reload_ext watermark
%watermark -a "Falcon FutureTech Maverick Team" -vmp langchain,huggingface_hub

Author: Falcon FutureTech Maverick Team

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.16.1

langchain      : 0.0.324
huggingface_hub: 0.17.3

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 6.2.0-1015-azure
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [9]:
# get your Huggingface access token from https://huggingface.co/settings/tokens 🔑
from getpass import getpass
import os

HUGGINGFACE_API_TOKEN = getpass()
os.environ["HUGGINGFACE_API_TOKEN"] = HUGGINGFACE_API_TOKEN   

#### Let's use falcon-7b-instruct model from [Huggingface website](https://huggingface.co/tiiuae/falcon-7b-instruct)

In [10]:
from langchain import HuggingFaceHub

repo_id = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceHub(huggingfacehub_api_token=HUGGINGFACE_API_TOKEN, 
                     repo_id=repo_id, 
                     model_kwargs={"temperature":0.7, "max_new_tokens":700})

/workspaces/falcon-main/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [11]:
from langchain import PromptTemplate, LLMChain

template = """
You are a helpful AI assistant and provide the answer for the question asked politely.

{question}
Answer: Let's think step by step.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "How to cook Pizza ?"

In [12]:
responseText = llm_chain.run(question)

len(responseText)

print(responseText)


1. Preheat your oven at the desired temperature for the specific type of pizza you want to make.
2. Roll out your dough on a baking sheet or pizza stone.
3. Spread your favorite pizza sauce on the dough.
4. Add your desired amount of cheese.
5. Place your toppings on the pizza and bake for the amount of time recommended on the recipe.
6. Remove the pizza from the oven and let it cool before slicing. 
7. Enjoy your delicious pizza!

I hope this helps!


In [13]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)
model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

In [14]:
inputs = processor(text=responseText, return_tensors="pt")

print (inputs)

{'input_ids': tensor([[ 4,  3, 26,  4, 49, 13,  5, 11,  5,  7,  6,  4, 22,  8, 16, 13,  4,  8,
         27,  5,  9,  4,  7,  6,  4,  6, 11,  5,  4, 14,  5, 12, 10, 13,  5, 14,
          4,  6,  5, 18, 24,  5, 13,  7,  6, 16, 13,  5,  4, 19,  8, 13,  4,  6,
         11,  5,  4, 12, 24,  5, 17, 10, 19, 10, 17,  4,  6, 22, 24,  5,  4,  8,
         19,  4, 24, 10, 57, 57,  7,  4, 22,  8, 16,  4, 20,  7,  9,  6,  4,  6,
          8,  4, 18,  7, 28,  5, 26,  4,  3, 26,  4, 54,  8, 15, 15,  4,  8, 16,
          6,  4, 22,  8, 16, 13,  4, 14,  8, 16, 21, 11,  4,  8,  9,  4,  7,  4,
         25,  7, 28, 10,  9, 21,  4, 12, 11,  5,  5,  6,  4,  8, 13,  4, 24, 10,
         57, 57,  7,  4, 12,  6,  8,  9,  5, 26,  4,  3, 26,  4, 34, 24, 13,  5,
          7, 14,  4, 22,  8, 16, 13,  4, 19,  7, 27,  8, 13, 10,  6,  5,  4, 24,
         10, 57, 57,  7,  4, 12,  7, 16, 17,  5,  4,  8,  9,  4,  6, 11,  5,  4,
         14,  8, 16, 21, 11, 26,  4,  3, 26,  4, 33, 14, 14,  4, 22,  8, 16, 13,
          4, 1

In [15]:
# Let’s load such a speaker embedding from a dataset on the Hub. 

# the embeddings were obtained from the CMU ARCTIC dataset: 
# http://www.festvox.org/cmu_arctic/

# using this script: 
# https://huggingface.co/mechanicalsea/speecht5-vc/blob/main/manifest/utils/prep_cmu_arctic_spkemb.py

# but any X-Vector embedding should work.

from datasets import load_dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
len(embeddings_dataset)

7931

In [16]:
# The speaker embedding is a tensor of shape (1, 512). 
# This particular speaker embedding describes a female voice.
import torch

speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
print(inputs["input_ids"])

tensor([[ 4,  3, 26,  4, 49, 13,  5, 11,  5,  7,  6,  4, 22,  8, 16, 13,  4,  8,
         27,  5,  9,  4,  7,  6,  4,  6, 11,  5,  4, 14,  5, 12, 10, 13,  5, 14,
          4,  6,  5, 18, 24,  5, 13,  7,  6, 16, 13,  5,  4, 19,  8, 13,  4,  6,
         11,  5,  4, 12, 24,  5, 17, 10, 19, 10, 17,  4,  6, 22, 24,  5,  4,  8,
         19,  4, 24, 10, 57, 57,  7,  4, 22,  8, 16,  4, 20,  7,  9,  6,  4,  6,
          8,  4, 18,  7, 28,  5, 26,  4,  3, 26,  4, 54,  8, 15, 15,  4,  8, 16,
          6,  4, 22,  8, 16, 13,  4, 14,  8, 16, 21, 11,  4,  8,  9,  4,  7,  4,
         25,  7, 28, 10,  9, 21,  4, 12, 11,  5,  5,  6,  4,  8, 13,  4, 24, 10,
         57, 57,  7,  4, 12,  6,  8,  9,  5, 26,  4,  3, 26,  4, 34, 24, 13,  5,
          7, 14,  4, 22,  8, 16, 13,  4, 19,  7, 27,  8, 13, 10,  6,  5,  4, 24,
         10, 57, 57,  7,  4, 12,  7, 16, 17,  5,  4,  8,  9,  4,  6, 11,  5,  4,
         14,  8, 16, 21, 11, 26,  4,  3, 26,  4, 33, 14, 14,  4, 22,  8, 16, 13,
          4, 14,  5, 12, 10,

In [17]:
# At this point we already have enough inputs to generate a log mel spectrogram as an output, 
# you can do it like this:

spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)

In [18]:
print(spectrogram)

# This outputs a tensor of shape (140, 80) containing a log mel spectrogram.

# it may vary between runs as the speech decoder pre-net always applies dropout to the input sequence

# This adds a bit of random variability to the generated speech

# if we are looking to generate speech waveform, we need to specify a vocoder to use for the spectrogram to waveform conversion

# Conveniently, 🤗 Transformers offers a vocoder based on HiFi-GAN

# HiFi-GAN is a state-of-the-art generative adversarial network (GAN) designed for high-fidelity speech synthesis. 
# It is capable of generating high-quality and realistic audio waveforms from spectrogram inputs.

tensor([[-2.2915, -2.3303, -2.3641,  ..., -2.5673, -2.6669, -2.8737],
        [-2.0063, -2.0446, -2.0787,  ..., -2.1490, -2.2210, -2.4764],
        [-1.8492, -2.0000, -2.0446,  ..., -2.2749, -2.2821, -2.5942],
        ...,
        [-2.4372, -2.6248, -2.7316,  ..., -3.7129, -3.7651, -4.0196],
        [-2.4452, -2.6135, -2.7041,  ..., -3.7317, -3.7813, -4.0401],
        [-2.4532, -2.6176, -2.7110,  ..., -3.7353, -3.7887, -4.0448]])


In [19]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Now all you need to do is pass it as an argument when generating speech, 
# and the outputs will be automatically converted to the speech waveform.
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [20]:
from IPython.display import Audio
Audio(speech, rate=16000)